In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf

In [2]:
data = np.load('/work/n1000000_0804_all_flat.npz')
model = tf.keras.models.load_model('/work/ML4NO/ML/Regression/models_all/0804_delta1.h5')
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 244)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 244)               0         
_________________________________________________________________
normalization (Normalization (None, 244)               489       
_________________________________________________________________
dense (Dense)                (None, 32)                7840      
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               4224      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128)              

In [ ]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])
target = data['delta']

x_train = data_all[:10000]
y_train = target[:10000]
x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]
x_test = data_all[900000:]
y_test = target[900000:]

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
generate = 1
num_of_bins = len(x_train[0])
scale_steps = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]
before_triain_loss = []
after_train_loss = []

for scale in scale_steps:
    print("std = {} start".format(scale))
    x_train2_gen = np.reshape(np.random.normal(x_train2, np.sqrt(x_train2)*scale, size = (generate, len(x_train2), num_of_bins)), (generate*len(x_train2), num_of_bins))
    y_train2_gen = np.repeat(y_train2, generate)
    x_test_gen = np.reshape(np.random.normal(x_test, np.sqrt(x_test)*scale, size = (generate, len(x_test), num_of_bins)), (generate*len(x_test), num_of_bins))
    y_test_gen = np.repeat(y_test, generate)

    before_triain_loss.append(model.evaluate(x_test_gen, y_test_gen)[0])

    model.fit(x_train2_gen, y_train2_gen,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test_gen)[0])

In [ ]:
print(scale_steps)
print(before_triain_loss)
print(after_train_loss)

In [ ]:
# pre_test = model.predict(x_test_gen)

In [ ]:
# plt.hist(pre_test, histtype = 'step', bins = 100, label = 'prediction')
# plt.hist(y_test_gen, histtype = 'step', bins = 100, label = 'target')
# plt.xlabel('delta')
# plt.ylabel('number')
# plt.legend()
# plt.show()

In [ ]:
# plt.figure(figsize=(7,7))
# plt.plot(pre_test, y_test, '.', alpha = 0.05)
# plt.xlabel('prediction')
# plt.ylabel('target')
# plt.title('delta')

In [ ]:
modelname = '/work/ML4NO/ML/Regression/models_furthurTrain/0804_delta_1.h5'
if not os.path.isfile(modelname):
    model.save(modelname)

In [ ]:
scale_steps = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]

outfile = {'scale_steps': scale_steps, 'before_triain_loss': before_triain_loss, 'after_train_loss', :after_train_loss}
np.save(file = '/work/ML4NO/ML/Regression/models_furthurTrain/0804_delta_1_result.npy', arr = outfile)